In [1]:
import pandas as pd

# 일본, 불매, 일제, no japan, 일본여행, 노재팬
general = pd.read_csv('2019-09-21_tweets.csv')
# 아사히, 유니클로, 대마도, 규슈, 후쿠오카, 데상트, DHC, 캐논,  도요타
com_area = pd.read_csv('2019-09-21_tweets_company.csv')
tweet_data = pd.concat([general, com_area])
tweet_data = tweet_data.drop(['Unnamed: 0'], axis=1)
tweet_data.head(5)

,username,timestamp,retweets,likes,is_retweet,text
0,Esthersumi,2019-08-17 23:36:36,126,42,0,한국인과 결혼해서 광고도 찍고 방송에도 나오다 홀연히 사라진 일본인.. 그 후 혐한...
1,chunnha,2019-08-17 22:52:19,1,0,0,일본군 둥닝요새의 ‘위안부 참상’ 중국 헤이룽장성의 관동군 주둔지에서 고초 겪은 ...
2,hyeong_chun,2019-08-17 22:45:06,0,1,0,이형춘(Hyeong Chun Lee): 노인과 권위의 나라 일본(Japan cou...
3,woori153,2019-08-17 22:41:22,1,2,0,일본 패망 할때까지 쭈우욱 가야죠. #노 재팬
4,LoveMoon012,2019-08-17 22:11:04,17,12,0,문재인 대통령의 방향성이 옳다는 것을 다시 한 번 확인하게 되네. No 아베로 그칠...


검색된 내용 중 중복된 내용의 트윗을 제거한다.

- username과 text가 완전히 일치한다고 무조건 중복된 내용이라 볼 수 없다고 가정한다. 왜냐하면 같은 사람이 다른 날 같은 내용을 올리는 것은 관심도가 유지되고 있다는 것으로 가정하기 때문이다.
- username, timestamp, text가 완전히 일치한다면 같은 트윗이다. 날짜 이외의 시간은 의미없으므로 지워준다.

In [2]:
def modify_time(time):
    return time[:10]

# 날짜만 남김
tweet_data['timestamp'] = tweet_data['timestamp'].apply(modify_time)

print('중복 제거 전: {}'.format(len(tweet_data)))
tweet_data = tweet_data.drop_duplicates(['username', 'timestamp', 'text'], keep='last').reset_index(drop=True)
print('중복 제거 후: {}'.format(len(tweet_data)))

중복 제거 전: 128194
중복 제거 후: 125724


동일인물이 같은 날에 쓴 트윗들은 하나로 합쳐 한 사람의 의견이 여러 번 반영되지 않도록 한다.

In [34]:
processed_data = tweet_data[tweet_data.duplicated(['username']).apply(lambda x: not x)].reset_index(drop=True)
dup_username_tweets = tweet_data[tweet_data.duplicated(['username'])]

In [35]:
usernames = list(set(dup_username_tweets['username'].values))
for i, username in enumerate(usernames):
    # 진행개수 출력
    print('\r{} / {}'.format(i, 15595), end='')
    
    rows = dup_username_tweets[dup_username_tweets['username']==username]
    duplicated = rows.duplicated(['timestamp'])
    if duplicated.sum() == 0:
        continue
        
    processed_data = processed_data.append(rows[duplicated.apply(lambda x: not x)], ignore_index=True)
    rows = rows[duplicated]
    timestamps = list(set(rows['timestamp'].values))
    for timestamp in timestamps:
        likes = rows['likes'].sum()
        retweets = rows['retweets'].sum()
        new_text = ''
        for text in rows['text']:
            new_text = new_text + ' ' + text
            
        processed_data = processed_data.append({'username': username, 'timestamp':timestamp, 'retweets':retweets,
                               'likes':likes, 'is_retweet':0, 'text':new_text}, ignore_index=True)

15595 / 15595

In [41]:
print('처리 전 : {}'.format(len(tweet_data)))
print('처리 후 : {}'.format(len(processed_data)))
processed_data.tail(7)

처리 전 : 125724
처리 후 : 88917


,username,timestamp,retweets,likes,is_retweet,text
88910,Schalom1004,2019-07-22,294,167,0,쪽바리의 근성. 유니클로가 코리아 블랙프라이데이 세일 기간에 1만2900원짜리 T...
88911,Schalom1004,2019-07-10,5237,3435,0,일본도 한국제품 불매운동 하고 있음 일본제품 불매운동 소비제품 품목. 맥주> ...
88912,Schalom1004,2019-07-03,5237,3435,0,일본도 한국제품 불매운동 하고 있음 일본제품 불매운동 소비제품 품목. 맥주> ...
88913,Schalom1004,2019-07-12,5237,3435,0,일본도 한국제품 불매운동 하고 있음 일본제품 불매운동 소비제품 품목. 맥주> ...
88914,Schalom1004,2019-08-10,5237,3435,0,일본도 한국제품 불매운동 하고 있음 일본제품 불매운동 소비제품 품목. 맥주> ...
88915,Schalom1004,2019-07-04,5237,3435,0,일본도 한국제품 불매운동 하고 있음 일본제품 불매운동 소비제품 품목. 맥주> ...
88916,Schalom1004,2019-07-25,5237,3435,0,일본도 한국제품 불매운동 하고 있음 일본제품 불매운동 소비제품 품목. 맥주> ...


전처리 결과 저장

In [42]:
processed_data.to_csv('processed_tweetData.csv', index=False)

### 데이터 분석 --- 진행 중...

In [39]:
from soynlp.tokenizer import RegexTokenizer
from konlpy.tag import Twitter

tokenizer = RegexTokenizer()
tokened_content = tokenizer.tokenize(sample_tweet['text'])
print(len(tokened_content))
tokened_content   

43


['세상은',
 '넓고',
 '모지리들은',
 '많다',
 '.',
 '모지리',
 '질량보존의',
 '법칙도',
 '아니고',
 '...',
 '방사능이',
 '위험한데도',
 '그저',
 '좋다고',
 '놀러',
 '간다',
 '.',
 '일본에',
 '여행가고',
 '물건',
 '사주면',
 '그게',
 '다',
 '전쟁준비하는',
 '군자금',
 '이',
 '되는',
 '것도',
 '모르고',
 '그저',
 '좋',
 '단다',
 '.',
 '방사능에',
 '피폭되면',
 '자자손손',
 '어찌될지',
 '모르는데',
 '그래도',
 'NO',
 'JAPAN',
 '안할래',
 '?']

1. 일본 불매에 대해 긍정적인가 부정적인가
2. 일본에 대해 긍정적인가 부정적인가

In [54]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100, min_cohesion_forward=0.05)
word_extractor.train(com_area['text'])
words = word_extractor.extract()

training was done. used memory 1.205 Gbse memory 1.347 Gb
all cohesion probabilities was computed. # words = 6342
all branching entropies was computed # words = 171433
all accessor variety was computed # words = 171433


In [56]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

..     (2660, 0.918, 5.059)
유니클로     (28266, 0.947, 4.891)
...     (1783, 0.784, 5.055)
캐논     (12317, 0.934, 4.870)
!!     (353, 0.927, 4.825)
후쿠오카     (21461, 0.954, 4.762)
진짜     (4177, 0.659, 5.097)
아사히     (21035, 0.685, 4.836)
....     (596, 0.590, 4.851)
??     (321, 0.783, 4.542)
까지     (215, 0.358, 5.219)
ㅠㅠ     (2350, 0.854, 4.323)
!!!     (177, 0.682, 4.548)
DHC     (7229, 0.947, 4.211)
후쿠오카에서     (1330, 0.557, 4.706)
너무     (4098, 0.780, 4.340)
데상트     (1363, 0.726, 4.373)
근데     (2207, 0.629, 4.498)
때문에     (637, 0.559, 4.569)
.....     (251, 0.542, 4.548)
일본     (18512, 0.668, 4.326)
맥주     (3668, 0.947, 3.932)
했는데     (658, 0.458, 4.579)
???     (199, 0.697, 4.129)
ㅜㅜ     (485, 0.702, 4.098)
불매운동     (5294, 0.754, 4.021)
ㅋㅋ     (6102, 0.881, 3.860)
브랜드     (1709, 0.821, 3.907)
합니다.     (491, 0.684, 4.087)
갑자기     (503, 0.752, 3.990)
